<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

# OpenEnv GRPO with trl

In this tutorial notebook, we're going to use Oumi to train an agentic model on an [OpenEnv](https://github.com/meta-pytorch/OpenEnv) Echo reinforcement learning (RL) environment with the GRPO algorithm. To achieve this, we use the trl library by Hugging Face with a custom rollout function to interact with the vLLM server and OpenEnv environment. This notebook is derived from trl's [Echo environment example](https://github.com/huggingface/trl/blob/main/examples/scripts/openenv/echo.py).

# 📋 Prerequisites

❗**NOTICE:** This notebook needs to be running on a machine with at least two GPUs.

## Oumi Installation

First, let's install the latest versions of Oumi and OpenEnv. You can find more detailed instructions [here](https://oumi.ai/docs/en/latest/get_started/installation.html).

In [ ]:
!pip install uv && uv pip install "oumi[gpu] @ git+https://github.com/oumi-ai/oumi.git"
!uv pip install git+https://github.com/meta-pytorch/OpenEnv.git

In [1]:
import os
from pathlib import Path

tutorial_dir = "openenv_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Disable warnings from HF.

# Start OpenEnv and vLLM servers

We need to run 2 servers in addition to the trl trainer. The OpenEnv server receives actions from the LLM and returns the updated state and reward. The vLLM server is used for inference, and updates it weights over training with the updated model weights from the trainer. We start these with separate subprocesses.

In [2]:
%%writefile $tutorial_dir/start_openenv_server.py

import os
import subprocess
import sys
import threading
import time
from pathlib import Path

import requests


def stream_output(pipe, prefix=""):
    """Stream output lines from subprocess pipe to stdout."""
    for line in iter(pipe.readline, ""):
        print(f"{prefix}{line}", end="")
    pipe.close()


print("⚡ Starting FastAPI server for Echo Environment...")

work_dir = str(Path.cwd().parent.absolute())

server_process = subprocess.Popen(
    [
        sys.executable,
        "-m",
        "uvicorn",
        "envs.echo_env.server.app:app",
        "--host",
        "0.0.0.0",
        "--port",
        "8001",
    ],
    env={**os.environ, "PYTHONPATH": f"{work_dir}/src"},
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    cwd=work_dir,
)

# Start background threads to stream errors
threading.Thread(
    target=stream_output, args=(server_process.stderr, "🔥 [stderr] "), daemon=True
).start()

print("⏳ Waiting for server to start...")
time.sleep(5)

try:
    response = requests.get("http://0.0.0.0:8001/health", timeout=2)
    print("\n✅ Echo Environment server is running!")
except Exception as e:
    print(f"\n❌ Server failed to start: {e}")
    print("\n📋 Checking error output...")
    server_process.poll()
    if server_process.stderr:
        stderr = server_process.stderr.read()
        if stderr:
            print(stderr)
    raise

try:
    input("Press Enter to exit...\n")
finally:
    print("🛑 Stopping server...")
    server_process.terminate()
    server_process.wait()

Overwriting openenv_tutorial/start_openenv_server.py


In [3]:
import subprocess

# Start both servers in the background
server1 = subprocess.Popen(
    [
        "bash",
        "-c",
        (
            "CUDA_VISIBLE_DEVICES=0 trl vllm-serve "
            "--model Qwen/Qwen2.5-0.5B-Instruct "
            "--log-level warning "
            "--host 0.0.0.0 --port 8000"
        ),
    ]
)
server2 = subprocess.Popen(["python", f"{tutorial_dir}/start_openenv_server.py"])

print("Servers started. PIDs:", server1.pid, server2.pid)

Servers started. PIDs: 3787594 3787595


In [4]:
import time

import requests

URL = "http://0.0.0.0:8000/health"


def check_vllm_health():
    """Checks if the vLLM server is healthy."""
    try:
        response = requests.get(URL, timeout=3)
        if response.status_code == 200:
            print("✅ vLLM server is healthy!")
            return True
        else:
            print(f"⚠️ Server responded with {response.status_code}")
    except requests.RequestException as e:
        print(f"❌ Server not ready: {e}")
    return False


max_retries = 24
for attempt in range(1, max_retries + 1):
    if check_vllm_health():
        break
    time.sleep(5)
else:
    print(f"❌ Failed to start vLLM server after {max_retries} attempts.")

⚡ Starting FastAPI server for Echo Environment...
⏳ Waiting for server to start...
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d30b0f93950>: Failed to establish a new connection: [Errno 111] Connection refused'))
🔥 [stderr] INFO:     Started server process [3787596]
🔥 [stderr] INFO:     Waiting for application startup.
🔥 [stderr] INFO:     Application startup complete.
🔥 [stderr] INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)

✅ Echo Environment server is running!
Press Enter to exit...
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d30b0f9df50>: Failed to establish a new connection: [Errno 111] Connection refused'))
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max 

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-31 17:31:42 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=16384.
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d30b0fad190>: Failed to establish a new connection: [Errno 111] Connection refused'))
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d30b0fa44d0>: Failed to establish a new connection: [Errno 111] Connection refused'))
INFO 10-31 17:31:51 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:52 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:52 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, to

[W1031 17:31:53.898867831 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:53 [parallel_state.py:1165] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=3787957) WARNING 10-31 17:31:53 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:53 [gpu_model_ru

...

(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:53 [default_loader.py:268] Loading weights took 0.18 seconds
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:54 [gpu_model_runner.py:2392] Model loading took 0.9266 GiB and 0.487034 seconds
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d30b0f9d690>: Failed to establish a new connection: [Errno 111] Connection refused'))
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:58 [backends.py:539] Using cache directory: /home/wizeng/.cache/vllm/torch_compile_cache/5d31f4c583/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:58 [backends.py:550] Dynamo bytecode transform time: 3.53 s
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:59 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.361 s
(EngineCore_DP0 pid=3787957) INFO 10-31 17:31:59 [

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE):  81%|████████  | 54/67 [00:01<00:00, 38.49it/s]

❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d30b0faefd0>: Failed to establish a new connection: [Errno 111] Connection refused'))


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:01<00:00, 40.82it/s]


(EngineCore_DP0 pid=3787957) INFO 10-31 17:32:02 [gpu_model_runner.py:3118] Graph capturing finished in 2 secs, took 0.50 GiB
(EngineCore_DP0 pid=3787957) INFO 10-31 17:32:02 [gpu_worker.py:391] Free memory on device (78.59/79.19 GiB) on startup. Desired GPU memory utilization is (0.9, 71.27 GiB). Actual usage is 0.93 GiB for weight, 5.57 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.5 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=68779733708` to fit into requested memory, or `--kv-cache-memory=76635612672` to fully utilize gpu memory. Current kv cache memory in use is 69478085324 bytes.
(EngineCore_DP0 pid=3787957) INFO 10-31 17:32:02 [core.py:218] init engine (profile, create kv cache, warmup model) took 8.68 seconds
INFO 10-31 17:32:04 [llm.py:295] Supported_tasks: ['generate']
INFO 10-31 17:32:04 [__init__.py:36] No IOProcessor plugins requested by the model
✅ vLLM server is healthy!


# Train the model!

By providing a custom rollout function to interact with the OpenEnv and vLLM servers, we can use trl to do agentic GRPO training. We also need to provide a reward function that processes the reward value output by the environment.

The following script defines the custom rollout and reward functions and runs the trainer. We run it as a subprocess so that we can set `CUDA_VISIBLE_DEVICES` to not conflict with the vLLM server.

In [6]:
%%writefile $tutorial_dir/train.py

import requests
from envs.echo_env import EchoEnv
from envs.echo_env.models import EchoAction

from oumi.core.configs import TrainingConfig
from oumi.core.registry import RegistryType, register
from oumi.train import train


@register("env_reward", RegistryType.REWARD_FUNCTION)
def reward_from_env(completions, **kwargs):
    """Reward function that uses the environment reward."""
    # Extract environment rewards from kwargs (propagated via extra_fields)
    env_rewards = kwargs.get("env_reward", [])
    if env_rewards:
        return [float(reward) for reward in env_rewards]
    else:
        # Fallback if env_reward is not available
        return [0.0] * len(completions)


@register("echo_env_vllm_rollout", RegistryType.ROLLOUT_FUNCTION)
def echo_env_vllm_rollout(
    prompts: list[str], args, processing_class
) -> dict[str, list]:
    """Custom rollout function that generates completions via vLLM server and computes environment rewards.

    Args:
        prompts: List of prompts to generate from
        args: GRPOConfig containing all sampling parameters
        processing_class: Tokenizer/processor for decoding completions

    Returns:
        Dict containing prompt_ids, completion_ids, logprobs, and env_reward
    """  # noqa: E501
    # 1. Generate completions via vLLM inference server (running on port 8000)
    payload = {
        "prompts": prompts,
        "n": args.num_generations,
        "temperature": args.temperature,
        "top_p": args.top_p,
        "top_k": -1 if args.top_k is None else args.top_k,
        "min_p": 0.0 if args.min_p is None else args.min_p,
        "max_tokens": args.max_completion_length,
        "repetition_penalty": args.repetition_penalty,
    }
    response = requests.post("http://0.0.0.0:8000/generate/", json=payload)

    if response.status_code != 200:
        print(f"Error response: {response.text}")

    response.raise_for_status()
    result = response.json()

    completions_text = processing_class.batch_decode(
        result["completion_ids"], skip_special_tokens=True
    )

    # 2. Step through the environment to get rewards
    client = EchoEnv(base_url="http://0.0.0.0:8001")
    env_result = client.reset()
    env_rewards = []
    for msg in completions_text:
        env_result = client.step(EchoAction(message=msg))
        env_rewards.append(env_result.reward)

    # 3. Add environment rewards as extra field
    result["env_reward"] = env_rewards

    return result


config = TrainingConfig.from_yaml("openenv_tutorial/grpo_train.yaml")
train(config)

Overwriting openenv_tutorial/train.py


Finally, we define the YAML training config, and kick off training!

To enable logging to Weights and Biases, uncomment the relevant line in the config below, and make sure to [set up wandb](https://oumi.ai/docs/en/latest/development/dev_setup.html#optional-set-up-weights-and-biases) on your machine.

In [7]:
%%writefile $tutorial_dir/grpo_train.yaml

model:
  model_name: "Qwen/Qwen2-0.5B-Instruct"
  model_max_length: 2048
  torch_dtype_str: "bfloat16"
  attn_implementation: "sdpa"

data:
  train:
    datasets:
      - dataset_name: "trl-lib/ultrafeedback-prompt"
        split: "train"
        sample_count: 100

training:
  trainer_type: "TRL_GRPO"
  per_device_train_batch_size: 8
  gradient_accumulation_steps: 4

  reward_functions: ["env_reward"]

  ddp_find_unused_parameters: False
  optimizer: "adamw_torch_fused"

  grpo:
    use_vllm: True
    rollout_function: "echo_env_vllm_rollout"
    max_completion_length: 2048

  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32

  num_train_epochs: 1
  logging_steps: 1
  log_model_summary: False
  output_dir: "openenv_tutorial/echo_grpo"
  # Uncomment to enable wandb logging
  # enable_wandb: True

Overwriting openenv_tutorial/grpo_train.yaml


In [8]:
import os
import subprocess
import sys

env = {**os.environ, "CUDA_VISIBLE_DEVICES": "1"}
# Run the trainer as a subprocess to reinitialize CUDA with only the second GPU visible.
subprocess.run(
    [sys.executable, str(Path(tutorial_dir) / "train.py")], env=env, check=True
)

[2025-10-31 17:32:17,665][oumi][rank0][pid:3788270][MainThread][WARNING]][training_config.py:149] Ignored model.model_max_length=2048 parameter for trainer TrainerType.TRL_GRPO.
[2025-10-31 17:32:17,666][oumi][rank0][pid:3788270][MainThread][INFO]][train.py:117] Creating training.output_dir: openenv_tutorial/echo_grpo...
[2025-10-31 17:32:17,668][oumi][rank0][pid:3788270][MainThread][INFO]][train.py:119] Created training.output_dir absolute path: /home/wizeng/repos/oumi/notebooks/openenv_tutorial/echo_grpo
[2025-10-31 17:32:17,669][oumi][rank0][pid:3788270][MainThread][INFO]][train.py:117] Creating training.telemetry_dir: openenv_tutorial/echo_grpo/telemetry...
[2025-10-31 17:32:17,672][oumi][rank0][pid:3788270][MainThread][INFO]][train.py:119] Created training.telemetry_dir absolute path: /home/wizeng/repos/oumi/notebooks/openenv_tutorial/echo_grpo/telemetry
[2025-10-31 17:32:17,675][oumi][rank0][pid:3788270][MainThread][INFO]][torch_utils.py:80] Torch version: 2.8.0+cu128. NumPy vers

`torch_dtype` is deprecated! Use `dtype` instead!


[2025-10-31 17:32:19,171][oumi][rank0][pid:3788270][MainThread][INFO]][torch_utils.py:288] 
Model Parameters Summary:
🔢 Total     parameters: 494,032,768
🔗 Embedding parameters: 136,134,656
🎯 Trainable parameters: 494,032,768
🔒 Frozen    parameters: 0 (0.00%)

INFO 10-31 17:32:19 [__init__.py:216] Automatically detected platform cuda.
[2025-10-31 17:32:19,942][oumi][rank0][pid:3788270][MainThread][INFO]][torch_profiler_utils.py:164] PROF: Torch Profiler disabled!


/home/wizeng/repos/oumi/src/oumi/builders/training.py:70: UserWarning: You are importing from 'rollout_func', which is an experimental feature. This API may change or be removed at any time without prior notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  trainer = HuggingFaceTrainer(cls(*args, **kwargs, args=hf_args), processor)
The model is already on multiple devices. Skipping the move to device specified in `args`.


INFO 10-31 17:32:20 [__init__.py:1433] Found nccl from library libnccl.so.2
INFO 10-31 17:32:20 [pynccl.py:70] vLLM is using nccl==2.27.3
(EngineCore_DP0 pid=3787957) INFO 10-31 17:32:20 [__init__.py:1433] Found nccl from library libnccl.so.2
(EngineCore_DP0 pid=3787957) INFO 10-31 17:32:20 [pynccl.py:70] vLLM is using nccl==2.27.3
[2025-10-31 17:32:21,330][oumi][rank0][pid:3788270][MainThread][INFO]][device_utils.py:343] GPU Metrics Before Training: GPU runtime info: NVidiaGpuRuntimeInfo(device_index=0, device_count=2, used_memory_mb=75593.0, temperature=33, fan_speed=None, fan_speeds=None, power_usage_watts=123.946, power_limit_watts=700.0, gpu_utilization=0, memory_utilization=0, performance_state=0, clock_speed_graphics=1980, clock_speed_sm=1980, clock_speed_memory=2619).
[2025-10-31 17:32:21,330][oumi][rank0][pid:3788270][MainThread][INFO]][train.py:553] Training init time: 3.665s
[2025-10-31 17:32:21,330][oumi][rank0][pid:3788270][MainThread][INFO]][train.py:554] Starting trainin

...

100%|██████████| 25/25 [00:42<00:00,  1.64s/it]

{'loss': -0.0005, 'grad_norm': 3.0, 'learning_rate': 2.0000000000000003e-06, 'num_tokens': 245868.0, 'completions/mean_length': 256.0, 'completions/min_length': 256.0, 'completions/max_length': 256.0, 'completions/clipped_ratio': 1.0, 'completions/mean_terminated_length': 0.0, 'completions/min_terminated_length': 0.0, 'completions/max_terminated_length': 0.0, 'rewards/reward_from_env/mean': 199.96875, 'rewards/reward_from_env/std': 10.715091705322266, 'reward': 199.96875, 'reward_std': 8.32847785949707, 'frac_reward_zero_std': 0.0, 'sampling/sampling_logp_difference/mean': 0.05209578573703766, 'sampling/sampling_logp_difference/max': 1.2726020812988281, 'sampling/importance_sampling_ratio/min': 0.28010183572769165, 'sampling/importance_sampling_ratio/mean': 1.0139122009277344, 'sampling/importance_sampling_ratio/max': 1.518819808959961, 'entropy': 0.5849609375, 'clip_ratio/low_mean': 0.0001220703125, 'clip_ratio/low_min': 0.0001220703125, 'clip_ratio/high_mean': 0.0001220703125, 'clip_

100%|██████████| 25/25 [00:45<00:00,  1.83s/it]


{'train_runtime': 45.7829, 'train_samples_per_second': 2.184, 'train_steps_per_second': 0.546, 'train_loss': -0.023757427856326105, 'epoch': 1.0}
[2025-10-31 17:33:07,390][oumi][rank0][pid:3788270][MainThread][INFO]][train.py:561] Training is Complete.
[2025-10-31 17:33:07,391][oumi][rank0][pid:3788270][MainThread][INFO]][device_utils.py:343] GPU Metrics After Training: GPU runtime info: NVidiaGpuRuntimeInfo(device_index=0, device_count=2, used_memory_mb=75603.0, temperature=34, fan_speed=None, fan_speeds=None, power_usage_watts=124.915, power_limit_watts=700.0, gpu_utilization=0, memory_utilization=0, performance_state=0, clock_speed_graphics=1980, clock_speed_sm=1980, clock_speed_memory=2619).
[2025-10-31 17:33:07,391][oumi][rank0][pid:3788270][MainThread][INFO]][torch_utils.py:135] Peak GPU memory usage: 10.15 GB
[2025-10-31 17:33:07,391][oumi][rank0][pid:3788270][MainThread][INFO]][train.py:568] Saving final state...
[2025-10-31 17:33:07,395][oumi][rank0][pid:3788270][MainThread][I

CompletedProcess(args=['/home/wizeng/miniconda3/envs/openenv/bin/python', 'openenv_tutorial/train.py'], returncode=0)

If you enabled wandb logging, you should get a reward graph that looks like this. Even though the training duration was short, we can see that the model quickly learned to maximize the reward.


![Echo env reward graph](./assets/openenv_echo_reward.png)

# 🧭 What's Next?

Congrats on finishing this notebook! Feel free to check out our other [notebooks](https://github.com/oumi-ai/oumi/tree/main/notebooks) in the [Oumi GitHub](https://github.com/oumi-ai/oumi), and give us a star! You can also join the Oumi community over on [Discord](https://discord.gg/oumi).

📰 Want to keep up with news from Oumi? Subscribe to our [Substack](https://blog.oumi.ai/) and [Youtube](https://www.youtube.com/@Oumi_AI)!

⚡ Interested in building custom AI in hours, not months? Apply to get [early access](https://oumi-ai.typeform.com/early-access) to the Oumi Platform, or [chat with us](https://calendly.com/d/ctcx-nps-47m/chat-with-us-get-early-access-to-the-oumi-platform) to learn more!